In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 1.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285387 sha256=457e0b5cbcbf9e9f1664be86c09519bb79070afb168482ae4645e07c0418984a
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark


In [2]:
"""Task1.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/1HMePKJKMnucU_oP2Cjg30P05zUszynMf
"""

import os
import sys
import json
from pyspark import SparkContext


input_path = sys.argv[1]
output_path = sys.argv[2]
#input_path = "test_review.json"
#output_path = "output.json"



sc = SparkContext('local[*]', 'hw1_task1')
reviews_rdd = sc.textFile(input_path).map(lambda line: json.loads(line))

#A
total_reviews = reviews_rdd.count()
#B
reviews_2018 = reviews_rdd.filter(lambda line: line['date'][:4] == '2018').count()
#C
distinct_users = reviews_rdd.map(lambda line: line['user_id']).distinct().count()
#D
top10_users = reviews_rdd.map(lambda line: (line['user_id'], 1)).reduceByKey(lambda x, y: x+y).sortBy(lambda x: (-x[1], x[0])).take(10)
#E
distinct_business = reviews_rdd.map(lambda line: line['business_id']).distinct().count()
#F
top10_business = reviews_rdd.map(lambda line: (line['business_id'], 1)).reduceByKey(lambda x, y: x+y).sortBy(lambda x: (-x[1], x[0])).take(10)

output = {'n_review': total_reviews,
          'n_review_2018': reviews_2018,
          'n_user': distinct_users,
          'top10_user': top10_users,
          'n_business': distinct_business,
          'top10_business': top10_business}


with open(output_path, 'w') as output_file:
    json.dump(output, output_file)

sc.stop()

In [ ]:
#=========== TASK 2 ==================

In [ ]:
# -*- coding: utf-8 -*-
"""task2.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/1XdKzKxMnXxOnNB6iS7Juode3PIItDnTa
"""

import os
import sys
import json
from pyspark import SparkContext
import random
import time


input_path = sys.argv[1]
output_path = sys.argv[2]
n_partition = sys.argv[3]
#input_path = "test_review.json"
#output_path = "output2.json"
#n_partition = 2


sc = SparkContext('local[*]', 'hw1_task2')
reviews_rdd = sc.textFile(input_path).map(lambda line: json.loads(line))

result = {}
result["default"] = {}
result["customized"] = {}

def get_partition(x):
    return (len(x[0]) + random.randint(0, 1000)) % int(n_partition)

#default
default_start = time.time()
default_rdd = reviews_rdd.map(lambda row: (row['business_id'], 1))
default_partitions = default_rdd.getNumPartitions()
default_n_items = default_rdd.mapPartitions(lambda iterator: [len(list(iterator))], True).collect()

d_temp = default_rdd.reduceByKey(lambda x,y: x+y).sortBy(lambda x: x[1], ascending=False).map(lambda line: [line[0], line[1]]).take(10)
default_exe_time = time.time() - default_start

#Customized
custom_start = time.time()
customized_rdd = reviews_rdd.map(lambda x: (x['business_id'], 1)).partitionBy(int(n_partition), lambda x: ord(x[0][0]) % int(n_partition)).cache()
customized_partitions = customized_rdd.getNumPartitions()
customized_n_items = customized_rdd.mapPartitions(lambda iterator: [len(list(iterator))], True).collect()
c_temp = customized_rdd.reduceByKey(lambda x,y: x+y).sortBy(lambda x: x[1], ascending=False).map(lambda line: [line[0], line[1]]).take(10)
customized_exe_time = time.time() - custom_start


output = {'default': {'n_partition': default_partitions,
                      'n_items': default_n_items,
                      'exe_time': default_exe_time},
          'customized': {'n_partition': customized_partitions,
                         'n_items': customized_n_items,
                         'exe_time': customized_exe_time}}

with open(output_path, 'w') as output_file:
      json.dump(output, output_file)

sc.stop()

In [ ]:
#=========== TASK 3 ==================

In [ ]:
# -*- coding: utf-8 -*-
"""task3.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/18_I3iYXPb0MDSEy-R_qprzXwC0vUJwHb
"""

import json
from pyspark import SparkContext
import os
import sys
import time

review_path = sys.argv[1]
business_path = sys.argv[2]
output_path_a = sys.argv[3]
output_path_b = sys.argv[4]
#review_path = "test_review.json"
#business_path = "business.json"
#output_path_a = "output3a.txt"
#output_path_b = "output3b.json"



sc = SparkContext('local[*]', 'task3')


review_rdd = sc.textFile(review_path).map(lambda line: json.loads(line))
business_rdd = sc.textFile(business_path).map(lambda line: json.loads(line))

review = sc.textFile(review_path).map(json.loads).map(lambda x: (x['business_id'], x['stars']))
business = sc.textFile(business_path).map(json.loads).map(lambda x: (x['business_id'], x['city']))

data = review.join(business).map(lambda x: (x[1][1], x[1][0])).groupByKey().map(lambda x: (x[0], sum(x[1]) / len(x[1])))


data_temp = (
    review_rdd
    .map(lambda review: (review["business_id"], review["stars"]))
    .join(business_rdd.map(lambda business: (business["business_id"], business["city"])))
    .values()  # Get the (stars, city) pairs
    .groupByKey()
    .mapValues(lambda stars: sum(stars) / len(stars))  # Calculate average stars for each city
)

#data_a = sorted(data.collect(), key=lambda x: (-x[1], x[0]))


m1_start = time.time()
data_2 = data.collect()
data_2.sort(key=lambda x: (-x[1], x[0]))

print([s for s in data_2[:10]])
m1_time = time.time() - m1_start

with open(output_path_a, "w") as output_file_a:
  output_file_a.write("city,stars\n")
  #for city, stars in data.sortBy(lambda x: (-x[1], x[0])).collect():
        #output_file_a.write(f"{city},{stars}\n")
  for line in data_2:
    output_file_a.write(line[0]+","+str(line[1])+"\n")

m2_start = time.time()
print(data.takeOrdered(10, key=lambda x: (-x[1], x[0])))
m2_time = time.time() - m2_start

output = {'m1': m1_time,
          'm2': m2_time,
          'reason': "Sorting data in python returns a part of the RDD, whereas using spark the entire RDD is returened which is a more expensive operation"
}

with open(output_path_b, 'w') as output_file_b:
  json.dump(output, output_file_b)

sc.stop()